# Metadata

Analyze the angle metadata for Binaural-MUSDB.

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
from IPython.display import display, Audio

## Plotting Parameters

In [ ]:
PLOT_DIR = '../data/plots'

In [ ]:
plt.style.use('petroff10') # 'tableau-colorblind10'

In [ ]:
COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
STEMS = ['bass', 'drums', 'other', 'vocals']
DATASETS = ['standard', 'random', 'stereo']
MODELS = ['htdemucs', 'spleeter', 'umxhq']

In [ ]:
DATASET_PALETTE = dict(zip(DATASETS, COLORS[:3]))
STEM_PALETTE = dict(zip(STEMS, COLORS[3:7]))
MODEL_PALETTE = dict(zip(MODELS, COLORS[7:]))

### Dataset

In [ ]:
DATA_DIR = '../data/binaural_musdb18/test/'

In [ ]:
file_list = [f for f in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, f))]

In [ ]:
song = []
source = []
angle = []

for file in file_list:
    with open(os.path.join(DATA_DIR, file, 'metadata.json'), 'rb') as f:
        data = json.load(f)
        for k, v in data.items():
            if v > 90:
                v = v - 360
            song.append(file)
            source.append(k)
            angle.append(v)

In [ ]:
angles = pd.DataFrame({'title': song, 'source': source, 'random_angle': angle})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True, sharex=True, sharey=True)

sns.histplot(x='random_angle', hue='source', multiple='stack',
             data=angles, palette=STEM_PALETTE, binwidth=10, binrange=[-90, 100], ax=ax)

_legend = ax.legend_
h = _legend.legend_handles
l = [t.get_text() for t in _legend.get_texts()]

ax.set_xticks([-90, -60, -30, 0, 30, 60, 90],
               ['-90°', '-60°', '-30°', '0°', '30°', '60°', '90°'])
ax.get_legend().remove()
ax.set(ylabel='Count', xlabel='$\Theta$')

fig.legend(h, l, loc='upper center', fontsize=9, ncols=4, bbox_to_anchor=(0.55, 1.1))

plt.savefig('../data/plots/binaural_musdb_angles_bins.pdf', bbox_inches='tight')
plt.show()

### Export Metadata

In [ ]:
angles.to_csv('../data/binaural_musdb18/metadata.csv')